# Collecting all the ETL/TASBE data from CP1

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import glob
import os
import warnings
import re
import logging

from glob import glob
from copy import deepcopy
from numbers import Number

import numpy as np
import pandas as pd

import data_helpers

from xplan_experiment_analysis import read_credentials

import collect_cp1_data

In [3]:
DATA_DIR = os.path.expanduser("/work/projects/SD2E-Community/prod/data/")
assert os.path.exists(DATA_DIR)

# Get paths to all of the CP1 Transcriptic data from MongoDB

In [4]:
experiment_data_map = collect_cp1_data.main()

Found job but analytics is not on experiment.transcriptic.r1c7cpt3djxuj_r1c7fex29q6t2, continuing
Found job for experiment.transcriptic.r1cdfr5gjs72s_r1cdk2gjusax6
Archive path /products/v2/106de934ea535a5aaeb90d38ba9c6191/w3QXjO9JwVdeZ2X2Z8Aze0vo/PAV9DbgGVZp63W9WvEPEd5J9
Did not find job(s) for experiment.transcriptic.r1bcc65suuqrj
Found job for experiment.uw_biofab.17413
Archive path /products/v2/106de934ea535a5aaeb90d38ba9c6191/jD5rn21KR8ORy6jLlvkqKn0Q/0pE3YyWBq1W6KJxkPnz63eLG
Found job for experiment.transcriptic.r1ccqfbz332u8_r1cctaf8r32rc
Archive path /products/v2/106de934ea535a5aaeb90d38ba9c6191/PAVXyWovz3DZGLPk8QlpLgRe/Ylv1DXKkLQOn66ne31wGk5vZ
Did not find job(s) for experiment.transcriptic.r1bbq4mr76ngd
Found job for experiment.uw_biofab.10843
Archive path /products/v2/106de934ea535a5aaeb90d38ba9c6191/LYNodj5RYpeKlBPDQ4YL84eE/rj884DqD0wor8J2pJxEZygPJ
Found job for experiment.uw_biofab.10545
Archive path /products/v2/106de934ea535a5aaeb90d38ba9c6191/NDOjBQvD2zpGDX52G4reA3gd/5pV

Did not find job(s) for experiment.transcriptic.r1c9pjzjs2c9k_r1c9wva2tpwj7
Found job for experiment.uw_biofab.18598
Archive path /products/v2/106de934ea535a5aaeb90d38ba9c6191/y3AEVd5lYZArKjooXeW4qDBO/NDOEvzJDj0yDZYv5l9gO2qnr
Found job for experiment.uw_biofab.20250
Archive path /products/v2/106de934ea535a5aaeb90d38ba9c6191/xydv4pJvYwEpxW2Y3bo3r82r/Dbp5yr4qwWw6zRxwBZj5o5rG
Found job for experiment.uw_biofab.15984
Archive path /products/v2/106de934ea535a5aaeb90d38ba9c6191/y3lWwkVE8LgZkrE2oEWY4e3Q/PAVDq2bbGqEEGVEWkG6jdLJe
Found job for experiment.transcriptic.r1c5va879uaex_r1c639xp952g4
Archive path /products/v2/106de934ea535a5aaeb90d38ba9c6191/NDXo3on4VYG8K3KGl2O48j6r/YlrDPkN1PGW8j63gAZWxONwz
Found job but analytics is not on experiment.uw_biofab.25985, continuing
Found job but analytics is not on experiment.transcriptic.r1c84xvquwxth_r1c8ek7vcmdxh, continuing
Did not find job(s) for experiment.transcriptic.r1bfawa87byfb
Found job for experiment.transcriptic.r1btfp5k2edgn_r1btpym75nsdh


# Example data file to check format...

In [5]:
expt, fn = next(iter(experiment_data_map.items()))
print(expt)
print(fn)
print(fn[1:])
apath = os.path.join(DATA_DIR, fn[1:])
assert os.path.exists(apath) and os.path.isdir(apath)
#os.listdir(apath)
point_clouds = glob(os.path.join(apath, 'output/*PointCloud.csv'))
print(point_clouds[0])
tmp_df = pd.read_csv(point_clouds[0])
tmp_df.head()

experiment.transcriptic.r1cdfr5gjs72s_r1cdk2gjusax6
/products/v2/106de934ea535a5aaeb90d38ba9c6191/w3QXjO9JwVdeZ2X2Z8Aze0vo/PAV9DbgGVZp63W9WvEPEd5J9
products/v2/106de934ea535a5aaeb90d38ba9c6191/w3QXjO9JwVdeZ2X2Z8Aze0vo/PAV9DbgGVZp63W9WvEPEd5J9
/work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/w3QXjO9JwVdeZ2X2Z8Aze0vo/PAV9DbgGVZp63W9WvEPEd5J9/output/UW-BIOFAB-YEAST-119677_00003_1_B2_PointCloud.csv


,FSC_A_a.u.,SSC_A_a.u.,BL1_A_MEFL,RL1_A_a.u.
0,77155.0,277859.0,28055.88,57.51
1,125618.0,411984.0,60603.74,57.51
2,152656.0,508060.0,64284.06,57.51
3,185903.0,414327.0,50310.34,57.51
4,121899.0,364067.0,41397.05,57.51


In [6]:
tmp_df.describe()

,FSC_A_a.u.,SSC_A_a.u.,BL1_A_MEFL,RL1_A_a.u.
count,9102.000000,9102.000000,9102.000000,9.102000e+03
mean,145094.274445,445004.296199,62104.918891,9.000735e+03
std,48976.547027,171974.653342,33428.726319,4.919291e+04
min,57303.000000,141412.000000,57.510000,5.751000e+01
25%,108359.000000,310030.250000,38061.760000,5.751000e+01
50%,138744.000000,422190.000000,56578.390000,5.751000e+01
75%,173793.000000,560143.250000,80903.020000,1.550330e+03
max,283173.000000,891916.000000,358192.380000,1.274367e+06


In [7]:
tmp_df['BL1_A_MEFL'].describe()

count      9102.000000
mean      62104.918891
std       33428.726319
min          57.510000
25%       38061.760000
50%       56578.390000
75%       80903.020000
max      358192.380000
Name: BL1_A_MEFL, dtype: float64

In [8]:
plate = re.match('experiment.transcriptic.(.*)$', expt).group(1)
plate

'r1cdfr5gjs72s_r1cdk2gjusax6'

In [9]:
m = re.match(".*_([A-H])([0-9]*)_PointCloud.csv$", point_clouds[0])
col = m.group(1)
row =  "%02d" % 1 #int(m.group(2))
well = col + row
well

'B01'

In [10]:
new_row = tmp_df['BL1_A_MEFL'].describe()
new_row['well'] = well
new_row['plate'] = plate
new_row

count                         9102.0
mean                    62104.918891
std                     33428.726319
min                            57.51
25%                         38061.76
50%                         56578.39
75%                         80903.02
max                        358192.38
well                             B01
plate    r1cdfr5gjs72s_r1cdk2gjusax6
Name: BL1_A_MEFL, dtype: object

# Collect the data from each of the files in the table

In [19]:
class UnexpectedFileName(Exception):
    pass

def file_to_row(expt_id: str, filename: str, skip_unknown: bool = True) -> pd.Series:
    """
    Collect summary statistics for `expt_id` from the CSV file named `filename`.
    Drop the file if it doesn't match the expected pattern and `skip_unknown` is
    True.
    """
    logger = logging.getLogger("collect_all_etl")
    plate = re.match('experiment.transcriptic.(.*)$', expt_id).group(1)
    m = re.match(".*_([A-H])([0-9]*)_PointCloud.csv$", filename)
    tmp_df = pd.read_csv(filename)
    new_row = tmp_df['BL1_A_MEFL'].describe()
    if m is None:
        well = np.nan
        plate = np.nan
        
        if skip_unknown:
            raise UnexpectedFileName(f"Dropping file {filename}")
        else:
            logger.info("%s is not of expected shape.", filename)
    else:
        col = m.group(1)
        row =  "%02d" % int(m.group(2))
        well = col + row
    
    new_row['well'] = well
    new_row['plate'] = plate
    new_row['experiment_id'] = expt_id
    return new_row

In [20]:
file_to_row(expt, point_clouds[0])

count                                                       9102.0
mean                                                  62104.918891
std                                                   33428.726319
min                                                          57.51
25%                                                       38061.76
50%                                                       56578.39
75%                                                       80903.02
max                                                      358192.38
well                                                           B02
plate                                  r1cdfr5gjs72s_r1cdk2gjusax6
experiment_id    experiment.transcriptic.r1cdfr5gjs72s_r1cdk2gj...
Name: BL1_A_MEFL, dtype: object

In [21]:
file_to_row(expt, point_clouds[0], False)

count                                                       9102.0
mean                                                  62104.918891
std                                                   33428.726319
min                                                          57.51
25%                                                       38061.76
50%                                                       56578.39
75%                                                       80903.02
max                                                      358192.38
well                                                           B02
plate                                  r1cdfr5gjs72s_r1cdk2gjusax6
experiment_id    experiment.transcriptic.r1cdfr5gjs72s_r1cdk2gj...
Name: BL1_A_MEFL, dtype: object

In [58]:
from typing import List, Dict
def collect_all_data(experiment_data_map: Dict[str, str], drop_unknown: bool = True) -> pd.DataFrame:
    logger = logging.getLogger("collect_all_etl")
    all_rows: List[pd.Series] = []
    for expt, dirname in experiment_data_map.items():
        if not re.match('experiment\.transcriptic\..*', expt):
            logger.info(f"Skipping non-transcriptic data file {dirname} for {expt}")
            continue
        apath = os.path.join(DATA_DIR, dirname[1:])
        point_clouds = glob(os.path.join(apath, 'output/*PointCloud.csv'))
        for point_cloud in point_clouds:
            try:
                all_rows.append(file_to_row(expt, point_cloud, drop_unknown))
            except UnexpectedFileName as e:
                logger.error(e)
            except pd.errors.ParserError as e:
                logger.error(f"Parser error reading {point_cloud}\n{e}")
    df = pd.DataFrame(all_rows)
    return df.rename(columns={
                              'mean': 'mean_GFP_MEFL', 
                              'std': 'std_GFP_MEFL', 
                              'min': 'min_GFP_MEFL',
                              'max': 'max_GFP_MEFL',
                              '25%': '25%_GFP_MEFL',
                              '50%': '50%_GFP_MEFL',
                              '75%': '75%_GFP_MEFL' 
                             }).reset_index(drop=True).reindex(columns=['plate', 'well', 'count', 'mean_GFP_MEFL', 'std_GFP_MEFL', 'min_GFP_MEFL',
       '25%_GFP_MEFL', '50%_GFP_MEFL', '75%_GFP_MEFL', 'max_GFP_MEFL', 'experiment_id'])

In [59]:
full_df = collect_all_data(experiment_data_map)

Dropping file /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/lAVV1QqYnzOxykb01xpdXL40/qRy35P1yDNxjrBgKrg8yXpex/output/wt-control-1_00003_6_PointCloud.csv
Dropping file /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/lAVV1QqYnzOxykb01xpdXL40/qRy35P1yDNxjrBgKrg8yXpex/output/sampletranscripticaq1bsxppy4nkt9experimenttranscripticr1bsmgdayg2yq_r1bsu7tb7bsuk_PointCloud.csv
Dropping file /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/lAVV1QqYnzOxykb01xpdXL40/qRy35P1yDNxjrBgKrg8yXpex/output/sampletranscripticaq1bsxppy5ndj6experimenttranscripticr1bsmgdayg2yq_r1bsu7tb7bsuk_PointCloud.csv
Dropping file /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/lAVV1QqYnzOxykb01xpdXL40/qRy35P1yDNxjrBgKrg8yXpex/output/sampletranscripticaq1bsxppy3xkupexperimenttranscripticr1bsmgdayg2yq_r1bsu7tb7bsuk_PointCloud.csv
Dropping file /work/projects/SD2E-Community/prod/

Dropping file /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/lAVV1QqYnzOxykb01xpdXL40/qRy35P1yDNxjrBgKrg8yXpex/output/sampletranscripticaq1bsxppy4cszzexperimenttranscripticr1bsmgdayg2yq_r1bsu7tb7bsuk_PointCloud.csv
Dropping file /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/lAVV1QqYnzOxykb01xpdXL40/qRy35P1yDNxjrBgKrg8yXpex/output/sampletranscripticaq1bsxppy4zkskexperimenttranscripticr1bsmgdayg2yq_r1bsu7tb7bsuk_PointCloud.csv
Dropping file /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/lAVV1QqYnzOxykb01xpdXL40/qRy35P1yDNxjrBgKrg8yXpex/output/wt-control-1_75e-05_2_PointCloud.csv
Dropping file /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/lAVV1QqYnzOxykb01xpdXL40/qRy35P1yDNxjrBgKrg8yXpex/output/6390_75e-05_1_PointCloud.csv
Dropping file /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/lAVV1QqYnzOxykb01

Dropping file /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/lAVV1QqYnzOxykb01xpdXL40/qRy35P1yDNxjrBgKrg8yXpex/output/sampletranscripticaq1bsxppy52pvnexperimenttranscripticr1bsmgdayg2yq_r1bsu7tb7bsuk_PointCloud.csv
Dropping file /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/lAVV1QqYnzOxykb01xpdXL40/qRy35P1yDNxjrBgKrg8yXpex/output/sampletranscripticaq1bsxppy4zksnexperimenttranscripticr1bsmgdayg2yq_r1bsu7tb7bsuk_PointCloud.csv
Dropping file /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/lAVV1QqYnzOxykb01xpdXL40/qRy35P1yDNxjrBgKrg8yXpex/output/sampletranscripticaq1bsxppy5j494experimenttranscripticr1bsmgdayg2yq_r1bsu7tb7bsuk_PointCloud.csv
Dropping file /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/lAVV1QqYnzOxykb01xpdXL40/qRy35P1yDNxjrBgKrg8yXpex/output/NOR-00-Control___PointCloud.csv
Dropping file /work/projects/SD2E-Community/prod/data

Dropping file /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/lAVV1QqYnzOxykb01xpdXL40/qRy35P1yDNxjrBgKrg8yXpex/output/sampletranscripticaq1bsxppy3vdnjexperimenttranscripticr1bsmgdayg2yq_r1bsu7tb7bsuk_PointCloud.csv
Dropping file /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/lAVV1QqYnzOxykb01xpdXL40/qRy35P1yDNxjrBgKrg8yXpex/output/6388_75e-05_1_PointCloud.csv
Dropping file /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/lAVV1QqYnzOxykb01xpdXL40/qRy35P1yDNxjrBgKrg8yXpex/output/6391_00003_1_PointCloud.csv
Dropping file /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/lAVV1QqYnzOxykb01xpdXL40/qRy35P1yDNxjrBgKrg8yXpex/output/sampletranscripticaq1bsxppy48dmvexperimenttranscripticr1bsmgdayg2yq_r1bsu7tb7bsuk_PointCloud.csv
Dropping file /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/lAVV1QqYnzOxykb01xpdXL40/q

Dropping file /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/lAVV1QqYnzOxykb01xpdXL40/qRy35P1yDNxjrBgKrg8yXpex/output/wt-control-1_000015_2_PointCloud.csv
Dropping file /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/lAVV1QqYnzOxykb01xpdXL40/qRy35P1yDNxjrBgKrg8yXpex/output/sampletranscripticaq1bsxppz8mj5hexperimenttranscripticr1bsmgdayg2yq_r1bsu7tb7bsuk_PointCloud.csv
Dropping file /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/lAVV1QqYnzOxykb01xpdXL40/qRy35P1yDNxjrBgKrg8yXpex/output/6390_75e-05_2_PointCloud.csv
Dropping file /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/lAVV1QqYnzOxykb01xpdXL40/qRy35P1yDNxjrBgKrg8yXpex/output/sampletranscripticaq1bsxppy4ez85experimenttranscripticr1bsmgdayg2yq_r1bsu7tb7bsuk_PointCloud.csv
Dropping file /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/lAVV1QqYnzOxykb01

...

Dropping file /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/zPzxEYrNp3RNynRqbqzjBBY1/v26A2vVZY809lYQrzY9k4JO2/output/sampletranscripticaq1bt28yz9dtwzexperimenttranscripticr1bsqtgkytgmn_r1bsxqhkq5at9_PointCloud.csv
Dropping file /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/zPzxEYrNp3RNynRqbqzjBBY1/v26A2vVZY809lYQrzY9k4JO2/output/sampletranscripticaq1bt28yz6kx7eexperimenttranscripticr1bsqtgkytgmn_r1bsxqhkq5at9_PointCloud.csv
Dropping file /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/zPzxEYrNp3RNynRqbqzjBBY1/v26A2vVZY809lYQrzY9k4JO2/output/8543_00003_6_PointCloud.csv
Dropping file /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/zPzxEYrNp3RNynRqbqzjBBY1/v26A2vVZY809lYQrzY9k4JO2/output/8545_00003_5_PointCloud.csv
Dropping file /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/zPzxEYrNp3RNynRqbqzjBBY1/v2

Dropping file /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/zPzxEYrNp3RNynRqbqzjBBY1/v26A2vVZY809lYQrzY9k4JO2/output/sampletranscripticaq1bt28yz7e8f7experimenttranscripticr1bsqtgkytgmn_r1bsxqhkq5at9_PointCloud.csv
Dropping file /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/zPzxEYrNp3RNynRqbqzjBBY1/v26A2vVZY809lYQrzY9k4JO2/output/8542_00003_1_PointCloud.csv
Dropping file /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/zPzxEYrNp3RNynRqbqzjBBY1/v26A2vVZY809lYQrzY9k4JO2/output/sampletranscripticaq1bt28yz9bmqwexperimenttranscripticr1bsqtgkytgmn_r1bsxqhkq5at9_PointCloud.csv
Dropping file /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/zPzxEYrNp3RNynRqbqzjBBY1/v26A2vVZY809lYQrzY9k4JO2/output/sampletranscripticaq1bt28yz8f5a4experimenttranscripticr1bsqtgkytgmn_r1bsxqhkq5at9_PointCloud.csv
Dropping file /work/projects/SD2E-Community/prod/data/pro

Dropping file /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/zPzxEYrNp3RNynRqbqzjBBY1/v26A2vVZY809lYQrzY9k4JO2/output/sampletranscripticaq1bt28yza989sexperimenttranscripticr1bsqtgkytgmn_r1bsxqhkq5at9_PointCloud.csv
Dropping file /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/zPzxEYrNp3RNynRqbqzjBBY1/v26A2vVZY809lYQrzY9k4JO2/output/8542_75e-05_5_PointCloud.csv
Dropping file /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/zPzxEYrNp3RNynRqbqzjBBY1/v26A2vVZY809lYQrzY9k4JO2/output/sampletranscripticaq1bt28yz7kqxbexperimenttranscripticr1bsqtgkytgmn_r1bsxqhkq5at9_PointCloud.csv
Dropping file /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/zPzxEYrNp3RNynRqbqzjBBY1/v26A2vVZY809lYQrzY9k4JO2/output/sampletranscripticaq1bt28yz9rtwaexperimenttranscripticr1bsqtgkytgmn_r1bsxqhkq5at9_PointCloud.csv
Dropping file /work/projects/SD2E-Community/prod/data/pr

Dropping file /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/zPzxEYrNp3RNynRqbqzjBBY1/v26A2vVZY809lYQrzY9k4JO2/output/8543_000015_1_PointCloud.csv
Dropping file /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/zPzxEYrNp3RNynRqbqzjBBY1/v26A2vVZY809lYQrzY9k4JO2/output/sampletranscripticaq1bt28yz6xx6rexperimenttranscripticr1bsqtgkytgmn_r1bsxqhkq5at9_PointCloud.csv
Dropping file /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/zPzxEYrNp3RNynRqbqzjBBY1/v26A2vVZY809lYQrzY9k4JO2/output/sampletranscripticaq1bt28yz5y5evexperimenttranscripticr1bsqtgkytgmn_r1bsxqhkq5at9_PointCloud.csv
Dropping file /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/zPzxEYrNp3RNynRqbqzjBBY1/v26A2vVZY809lYQrzY9k4JO2/output/8543_00003_2_PointCloud.csv
Dropping file /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/zPzxEYrNp3RNynRqbqzjBBY1/v

Dropping file /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/zPzxEYrNp3RNynRqbqzjBBY1/v26A2vVZY809lYQrzY9k4JO2/output/sampletranscripticaq1bt28yz5smxpexperimenttranscripticr1bsqtgkytgmn_r1bsxqhkq5at9_PointCloud.csv
Dropping file /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/zPzxEYrNp3RNynRqbqzjBBY1/v26A2vVZY809lYQrzY9k4JO2/output/sampletranscripticaq1bt299srpj6texperimenttranscripticr1bsqtgkytgmn_r1bsxqhkq5at9_PointCloud.csv
Dropping file /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/zPzxEYrNp3RNynRqbqzjBBY1/v26A2vVZY809lYQrzY9k4JO2/output/8542_00003_6_PointCloud.csv
Dropping file /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/zPzxEYrNp3RNynRqbqzjBBY1/v26A2vVZY809lYQrzY9k4JO2/output/8543_000015_3_PointCloud.csv
Dropping file /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/zPzxEYrNp3RNynRqbqzjBBY1/v

In [60]:
full_df.head()

,plate,well,count,mean_GFP_MEFL,std_GFP_MEFL,min_GFP_MEFL,25%_GFP_MEFL,50%_GFP_MEFL,75%_GFP_MEFL,max_GFP_MEFL,experiment_id
0,r1cdfr5gjs72s_r1cdk2gjusax6,B02,9102.0,62104.918891,33428.726319,57.51,38061.760,56578.39,80903.02,358192.38,experiment.transcriptic.r1cdfr5gjs72s_r1cdk2gj...
1,r1cdfr5gjs72s_r1cdk2gjusax6,D01,17490.0,177119.717820,91700.505321,57.51,102927.460,157499.75,232486.34,634964.19,experiment.transcriptic.r1cdfr5gjs72s_r1cdk2gj...
2,r1cdfr5gjs72s_r1cdk2gjusax6,G01,15157.0,2919.289618,4139.485038,57.51,57.510,1258.53,4708.83,173428.65,experiment.transcriptic.r1cdfr5gjs72s_r1cdk2gj...
3,r1cdfr5gjs72s_r1cdk2gjusax6,E06,12997.0,58402.591344,35031.494043,57.51,32483.770,50425.35,76820.16,258478.62,experiment.transcriptic.r1cdfr5gjs72s_r1cdk2gj...
4,r1cdfr5gjs72s_r1cdk2gjusax6,E10,15624.0,36632.676914,22994.644878,57.51,19645.765,32052.48,49390.25,263309.05,experiment.transcriptic.r1cdfr5gjs72s_r1cdk2gj...


In [61]:
full_df.to_csv('TASBE_data.csv')

In [62]:
print(f"Filtering the rows w/o metadata, we have {full_df.shape[0]} rows/wells data")

Filtering the rows w/o metadata, we have 4028 rows/wells data


In [63]:
fuller_df = collect_all_data(experiment_data_map, False)
fuller_df.to_csv('TASBE_data_partial_metadata.csv')
print(f"Filtering the rows w/o metadata, we have {fuller_df.shape[0]} rows/wells data")

Parser error reading /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/zPo80jZyBvRoApQlwXZq9Lng/YlvvJDYE2l9YXLbrkKynBNOr/output/6391_00003_1_F9_PointCloud.csv
Error tokenizing data. C error: Expected 4 fields in line 590, saw 5

Parser error reading /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/zPo80jZyBvRoApQlwXZq9Lng/YlvvJDYE2l9YXLbrkKynBNOr/output/8544_00003_1_F5_PointCloud.csv
Error tokenizing data. C error: Expected 4 fields in line 353, saw 7

Parser error reading /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/zPo80jZyBvRoApQlwXZq9Lng/YlvvJDYE2l9YXLbrkKynBNOr/output/8542_00003_1_G1_PointCloud.csv
Error tokenizing data. C error: Expected 4 fields in line 235, saw 6

Parser error reading /work/projects/SD2E-Community/prod/data/products/v2/106de934ea535a5aaeb90d38ba9c6191/zPo80jZyBvRoApQlwXZq9Lng/YlvvJDYE2l9YXLbrkKynBNOr/output/5993_00003_1_H2_PointCloud.csv
Error tokenizi

Filtering the rows w/o metadata, we have 4788 rows/wells data


In [64]:
print(f"Rows without full metadata: {fuller_df.shape[0] - full_df.shape[0]}")

Rows without full metadata: 760


In [65]:
non_tasbe = pd.read_csv('new_accuracy_table.csv')

In [66]:
non_tasbe

,lab_id,strain_name,plate_id,well,count,count_live,experiment_id,filename,gate,gated_count,...,std_correct_low_threshold,std_correct_low_threshold_live,std_correct_threshold,std_correct_threshold_live,std_log_gfp,std_log_gfp_gated,std_log_gfp_gated_live,std_log_gfp_live,threshold,threshold_live
0,r1c5va879uaex_r1c639xp952g4,NOR-00-Control,0,C12,NaN,NaN,NaN,NaN,NaN,25743.0,...,NaN,NaN,NaN,NaN,NaN,0.286968,NaN,NaN,NaN,NaN
1,r1c5va879uaex_r1c639xp952g4,WT-Live-Control,0,A12,NaN,NaN,NaN,NaN,NaN,16591.0,...,NaN,NaN,NaN,NaN,NaN,0.469159,NaN,NaN,NaN,NaN
2,r1c5va879uaex_r1c639xp952g4,https://hub.sd2e.org/user/sd2e/design/UWBF_169...,0,B03,28297.0,24410.0,NaN,/work/05202/dbryce/xplan-reactor/fcs/YeastGate...,XOR,24715.0,...,0.002825,0.002889,0.002825,0.002889,0.981708,0.395890,0.395018,0.934420,5.257244,5.284602
3,r1c5va879uaex_r1c639xp952g4,https://hub.sd2e.org/user/sd2e/design/UWBF_169...,0,D03,27568.0,23178.0,NaN,/work/05202/dbryce/xplan-reactor/fcs/YeastGate...,XOR,23737.0,...,0.002714,0.002791,0.002714,0.002791,1.035838,0.422622,0.421670,0.980817,5.257244,5.284602
4,r1c5va879uaex_r1c639xp952g4,https://hub.sd2e.org/user/sd2e/design/UWBF_169...,0,A10,29862.0,15228.0,NaN,/work/05202/dbryce/xplan-reactor/fcs/YeastGate...,XOR,25483.0,...,0.002156,0.003665,0.002156,0.003665,0.872558,0.327265,0.327281,0.928051,5.257244,5.284602
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8721,r1dag7xpmgar9k_r1dakfqtk7hpzb,https://hub.sd2e.org/user/sd2e/design/UWBF_8544/1,0,D06,29960.0,28209.0,2019_07_01_14_23_28,/work/projects/SD2E-Community/prod/projects/sd...,NAND,NaN,...,0.000956,0.000980,0.000956,0.000980,0.750163,NaN,NaN,0.687830,6.375433,6.366930
8722,r1dag7xpmgar9k_r1dakfqtk7hpzb,https://hub.sd2e.org/user/sd2e/design/UWBF_8544/1,0,H11,29806.0,25017.0,2019_07_01_14_23_28,/work/projects/SD2E-Community/prod/projects/sd...,NAND,NaN,...,0.001797,0.002047,0.001797,0.002047,0.879183,NaN,NaN,0.710096,6.375433,6.366930
8723,r1dag7xpmgar9k_r1dakfqtk7hpzb,https://hub.sd2e.org/user/sd2e/design/UWBF_8545/1,0,A03,29836.0,26606.0,2019_07_01_14_23_28,/work/projects/SD2E-Community/prod/projects/sd...,NAND,NaN,...,0.001313,0.001402,0.001313,0.001402,0.852242,NaN,NaN,0.730193,6.375433,6.366930
8724,r1dag7xpmgar9k_r1dakfqtk7hpzb,https://hub.sd2e.org/user/sd2e/design/UWBF_8545/1,0,A10,29803.0,26441.0,2019_07_01_14_23_28,/work/projects/SD2E-Community/prod/projects/sd...,NAND,NaN,...,0.001461,0.001578,0.001461,0.001578,0.862770,NaN,NaN,0.739863,6.375433,6.366930


In [67]:
non_tasbe.shape[0]

8726

In [68]:
def drop_lab_prefix(ex_id: str) -> str:
    m = re.match('experiment.transcriptic.(.*)$', ex_id)
    return m.group(1)

drop_all_lab_prefix = np.vectorize(drop_lab_prefix)

In [69]:
full_df['lab_id'] = drop_all_lab_prefix(full_df['experiment_id'])

In [70]:
full_df.set_index(['lab_id', 'well'], inplace=True)
non_tasbe.set_index(['lab_id', 'well'], inplace=True)

In [71]:
joined = non_tasbe.join(full_df, rsuffix='_etl', how='inner')

In [72]:
joined.head()

strain_name  \
lab_id                      well                                                      
r1c5va879uaex_r1c639xp952g4 C12                                      NOR-00-Control   
                            A12                                     WT-Live-Control   
                            B03   https://hub.sd2e.org/user/sd2e/design/UWBF_169...   
                            D03   https://hub.sd2e.org/user/sd2e/design/UWBF_169...   
                            A10   https://hub.sd2e.org/user/sd2e/design/UWBF_169...   

                                  plate_id    count  count_live experiment_id  \
lab_id                      well                                                
r1c5va879uaex_r1c639xp952g4 C12          0      NaN         NaN           NaN   
                            A12          0      NaN         NaN           NaN   
                            B03          0  28297.0     24410.0           NaN   
                            D03          0  27568.0     23178.0           NaN   
                            A10          0  29862.0     15228.0           NaN   

                                                                           filename  \
lab_id                      well                                                      
r1c5va879uaex_r1c639xp952g4 C12                                                 NaN   
                            A12                                                 NaN   
                            B03   /work/05202/dbryce/xplan-reactor/fcs/YeastGate...   
                            D03   /work/05202/dbryce/xplan-reactor/fcs/YeastGate...   
                            A10   /work/05202/dbryce/xplan-reactor/fcs/YeastGate...   

                                 gate  gated_count  gated_live_count  \
lab_id                      well                                       
r1c5va879uaex_r1c639xp952g4 C12   NaN      25743.0               NaN   
                            A12   NaN      16591.0               NaN   
                            B03   XOR      24715.0           21777.0   
                            D03   XOR      23737.0           21954.0   
                            A10   XOR      25483.0           24836.0   

                                  growth_media_1  ...  \
lab_id                      well                  ...   
r1c5va879uaex_r1c639xp952g4 C12              NaN  ...   
                            A12              NaN  ...   
                            B03   standard_media  ...   
                            D03   standard_media  ...   
                            A10   standard_media  ...   

                                                        plate count_etl  \
lab_id                      well                                          
r1c5va879uaex_r1c639xp952g4 C12   r1c5va879uaex_r1c639xp952g4   16570.0   
                            A12   r1c5va879uaex_r1c639xp952g4   27090.0   
                            B03   r1c5va879uaex_r1c639xp952g4   14434.0   
                            D03   r1c5va879uaex_r1c639xp952g4   14698.0   
                            A10   r1c5va879uaex_r1c639xp952g4   14199.0   

                                  mean_GFP_MEFL  std_GFP_MEFL  min_GFP_MEFL  \
lab_id                      well                                              
r1c5va879uaex_r1c639xp952g4 C12    28119.796097  12040.251435         54.58   
                            A12     1233.086445   1768.071322         54.58   
                            B03     4296.246729   4190.802516         54.58   
                            D03     4118.489416   6975.492627         54.58   
                            A10    13979.866574   8425.377160         54.58   

                                  25%_GFP_MEFL  50%_GFP_MEFL 75%_GFP_MEFL  \
lab_id                      well                                            
r1c5va879uaex_r1c639xp952g4 C12       19517.27      27103.25    35780.740   
                            A12          54.58         54.58     199

In [73]:
joined.shape[0]

3779

In [74]:
joined['strain_name'].unique()

array(['NOR-00-Control', 'WT-Live-Control',
       'https://hub.sd2e.org/user/sd2e/design/UWBF_16967/1',
       'https://hub.sd2e.org/user/sd2e/design/UWBF_16968/1',
       'https://hub.sd2e.org/user/sd2e/design/UWBF_16969/1',
       'https://hub.sd2e.org/user/sd2e/design/UWBF_16970/1',
       'https://hub.sd2e.org/user/sd2e/design/UWBF_6388/1',
       'https://hub.sd2e.org/user/sd2e/design/UWBF_6389/1',
       'https://hub.sd2e.org/user/sd2e/design/UWBF_6390/1',
       'https://hub.sd2e.org/user/sd2e/design/UWBF_6391/1',
       'https://hub.sd2e.org/user/sd2e/design/UWBF_7299/1',
       'https://hub.sd2e.org/user/sd2e/design/UWBF_7300/1',
       'https://hub.sd2e.org/user/sd2e/design/UWBF_7373/1',
       'https://hub.sd2e.org/user/sd2e/design/UWBF_7374/1',
       'https://hub.sd2e.org/user/sd2e/design/UWBF_7375/1',
       'https://hub.sd2e.org/user/sd2e/design/UWBF_7376/1',
       'https://hub.sd2e.org/user/sd2e/design/UWBF_7377/1',
       'https://hub.sd2e.org/user/sd2e/design/UWBF_8

In [77]:
experimental_rows = joined.query("strain_name not in ['NOR-00-Control', 'WT-Live-Control', 'WT-Dead-Control']").shape[0]
control_rows = joined.query("strain_name  in ['NOR-00-Control', 'WT-Live-Control', 'WT-Dead-Control']").shape[0]
print(f"Experimental strain rows: {experimental_rows}; control rows: {control_rows}; total rows: {joined.shape[0]}")

Experimental strain rows: 3664; control rows: 115; total rows: 3779


In [78]:
joined.query("strain_name  in ['NOR-00-Control', 'WT-Live-Control', 'WT-Dead-Control']").head()

strain_name  plate_id  count  \
lab_id                      well                                     
r1c5va879uaex_r1c639xp952g4 C12    NOR-00-Control         0    NaN   
                            A12   WT-Live-Control         0    NaN   
r1c5va9wyf3mx_r1c63bjzu5ruy C12    NOR-00-Control         1    NaN   
                            B12   WT-Dead-Control         1    NaN   
                            A12   WT-Live-Control         1    NaN   

                                  count_live experiment_id  \
lab_id                      well                             
r1c5va879uaex_r1c639xp952g4 C12          NaN           NaN   
                            A12          NaN           NaN   
r1c5va9wyf3mx_r1c63bjzu5ruy C12          NaN           NaN   
                            B12          NaN           NaN   
                            A12          NaN           NaN   

                                                                           filename  \
lab_id                      well                                                      
r1c5va879uaex_r1c639xp952g4 C12                                                 NaN   
                            A12                                                 NaN   
r1c5va9wyf3mx_r1c63bjzu5ruy C12   /work/05202/dbryce/xplan-reactor/fcs/YeastGate...   
                            B12   /work/05202/dbryce/xplan-reactor/fcs/YeastGate...   
                            A12   /work/05202/dbryce/xplan-reactor/fcs/YeastGate...   

                                 gate  gated_count  gated_live_count  \
lab_id                      well                                       
r1c5va879uaex_r1c639xp952g4 C12   NaN      25743.0               NaN   
                            A12   NaN      16591.0               NaN   
r1c5va9wyf3mx_r1c63bjzu5ruy C12   NaN      26169.0           12350.0   
                            B12   NaN          NaN               NaN   
                            A12   NaN      17434.0            7987.0   

                                  growth_media_1  ...  \
lab_id                      well                  ...   
r1c5va879uaex_r1c639xp952g4 C12              NaN  ...   
                            A12              NaN  ...   
r1c5va9wyf3mx_r1c63bjzu5ruy C12   standard_media  ...   
                            B12   standard_media  ...   
                            A12   standard_media  ...   

                                                        plate count_etl  \
lab_id                      well                                          
r1c5va879uaex_r1c639xp952g4 C12   r1c5va879uaex_r1c639xp952g4   16570.0   
                            A12   r1c5va879uaex_r1c639xp952g4   27090.0   
r1c5va9wyf3mx_r1c63bjzu5ruy C12   r1c5va9wyf3mx_r1c63bjzu5ruy    1222.0   
                            B12   r1c5va9wyf3mx_r1c63bjzu5ruy     378.0   
                            A12   r1c5va9wyf3mx_r1c63bjzu5ruy    1381.0   

                                  mean_GFP_MEFL  std_GFP_MEFL  min_GFP_MEFL  \
lab_id                      well                                              
r1c5va879uaex_r1c639xp952g4 C12    28119.796097  12040.251435         54.58   
                            A12     1233.086445   1768.071322         54.58   
r1c5va9wyf3mx_r1c63bjzu5ruy C12    43069.402430  22570.072418         54.95   
                            B12     1459.573889   2849.322749         54.95   
                            A12     1476.603020   2971.526376         54.95   

                                  25%_GFP_MEFL  50%_GFP_MEFL 75%_GFP_MEFL  \
lab_id                      well                                            
r1c5va879uaex_r1c639xp952g4 C12       19517.27     27103.250   35780.7400   
                            A12          54.58        54.580    1998.5800   
r1c5va9wyf3mx_r1c63bjzu5ruy C12       28079.70     37448.975   55459.4250   
                            B12          54.95        54.950    1963.8525   
                            A12          54.95        5

In [80]:
fuller_df

,plate,well,count,mean_GFP_MEFL,std_GFP_MEFL,min_GFP_MEFL,25%_GFP_MEFL,50%_GFP_MEFL,75%_GFP_MEFL,max_GFP_MEFL,experiment_id
0,r1cdfr5gjs72s_r1cdk2gjusax6,B02,9102.0,62104.918891,33428.726319,57.51,38061.760,56578.390,80903.0200,358192.38,experiment.transcriptic.r1cdfr5gjs72s_r1cdk2gj...
1,r1cdfr5gjs72s_r1cdk2gjusax6,D01,17490.0,177119.717820,91700.505321,57.51,102927.460,157499.750,232486.3400,634964.19,experiment.transcriptic.r1cdfr5gjs72s_r1cdk2gj...
2,r1cdfr5gjs72s_r1cdk2gjusax6,G01,15157.0,2919.289618,4139.485038,57.51,57.510,1258.530,4708.8300,173428.65,experiment.transcriptic.r1cdfr5gjs72s_r1cdk2gj...
3,r1cdfr5gjs72s_r1cdk2gjusax6,E06,12997.0,58402.591344,35031.494043,57.51,32483.770,50425.350,76820.1600,258478.62,experiment.transcriptic.r1cdfr5gjs72s_r1cdk2gj...
4,r1cdfr5gjs72s_r1cdk2gjusax6,E10,15624.0,36632.676914,22994.644878,57.51,19645.765,32052.480,49390.2500,263309.05,experiment.transcriptic.r1cdfr5gjs72s_r1cdk2gj...
...,...,...,...,...,...,...,...,...,...,...,...
4783,NaN,NaN,281197.0,33.645462,87.256461,0.04,0.040,0.040,6.4200,6289.74,experiment.transcriptic.r1bsqtgkytgmn_r1bsxqhk...
4784,NaN,NaN,358369.0,73.647831,138.353912,0.04,0.040,0.040,88.8400,2878.84,experiment.transcriptic.r1bsqtgkytgmn_r1bsxqhk...
4785,NaN,NaN,317460.0,3183.799111,3025.022206,0.04,1003.780,2093.495,4332.5275,23425.23,experiment.transcriptic.r1bsqtgkytgmn_r1bsxqhk...
4786,NaN,NaN,290042.0,3307.803097,3066.628440,0.04,1077.850,2220.520,4497.1225,31727.53,experiment.transcriptic.r1bsqtgkytgmn_r1bsxqhk...


In [82]:
tmp = fuller_df.rename(columns={'plate': 'lab_id'}).set_index(['lab_id', 'well'])
outer_joined = non_tasbe.join(tmp, rsuffix='_etl', how='outer')

In [83]:
outer_joined.to_csv('accuracy_table_and_ETL.csv')

In [84]:
outer_joined.shape[0]

9735